In [24]:
# Path to ffmpeg
ffmpeg_path = '/usr/local/bin/ffmpeg'
videos_path = 'data/videos/'
audio_path = 'data/audio/'  # now defined dynamically
flac_path = 'data/flac/'

%matplotlib inline
import sys
import os.path
# Make sure ffmpeg is on the path so sk-video can find it
sys.path.append(os.path.dirname(ffmpeg_path))
import skvideo.io
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pafy
import soundfile as sf
import subprocess as sp
from IPython.display import Video

In [25]:
# Set output settings
audio_codec = 'flac'
audio_container = 'flac'
video_codec = 'h264'
video_container = 'mp4'

### Sorting out the segments csv

In [26]:
dataset = 'data/balanced_train_segments.csv'

In [27]:
car_passing_by = '/t/dd00134'

In [28]:
outside = '/t/dd00128'

In [29]:
car = '/m/0k4j'

In [30]:
outside_rural = '/t/dd00129'

In [31]:
# '/t/dd00128',"Outside, urban or manmade" # that will actually include engine sound

In [32]:
# 327,/m/0btp2,"Traffic noise, roadway noise" # that will actually include engine sound

In [33]:
# 348,/t/dd00066,"Medium engine (mid frequency)"
# 349,/t/dd00067,"Heavy engine (low frequency)"
# 350,/m/01h82_,"Engine knocking"
# 351,/t/dd00130,"Engine starting"
# 352,/m/07pb8fc,"Idling"
# 353,/m/07q2z82,"Accelerating, revving, vroom"
# 343,/m/02mk9,"Engine"
# 315,/m/0ltv,"Race car, auto racing"
# 316,/m/07r04,"Truck"
# 312,/m/07rknqz,"Skidding"
# 313,/m/0h9mv,"Tire squeal"
# 306,/m/012f08,"Motor vehicle (road)"
# 307,/m/0k4j,"Car"

In [34]:
def get_video_list(label):
    with open(dataset) as f:
        lines = f.readlines()
    dl_list = [line.strip().split(', ') for line in lines[3:]]
    dl_list = [i for i in dl_list if label in i[-1]]
    print('Videos available for this label: ', len(dl_list))
    print("")
    return dl_list

In [35]:
# _ = get_video_list(outside_rural)

In [36]:
# _ = get_video_list(car)

### We can load the AudioSet training set and grab a video to download:

In [37]:
# Select a YouTube video from the training set
def select_video(video_info):
    
    ytid, ts_start, ts_end, labels = video_info
    ts_start, ts_end = float(ts_start), float(ts_end)
    duration = ts_end - ts_start

    print("YouTube ID: " + ytid)
    print("Trim Window: ({}, {})".format(ts_start, ts_end))
    
    return ytid, ts_start, ts_end, duration

### After getting the YouTube ID and trim window for the video we want, we can use `pafy` to get the download URL to the best quality video and audio:

In [38]:
# Get the URL to the video page

def get_urls(ytid):
    video_page_url = 'https://www.youtube.com/watch?v={}'.format(ytid)

    # Get the direct URLs to the videos with best audio and with best video (with audio)
    video = pafy.new(video_page_url)

    best_video = video.getbestvideo()
    best_video_url = best_video.url
    # print("Video URL: " + best_video_url)

    best_audio = video.getbestaudio()
    best_audio_url = best_audio.url
    # print("Audio URL: " + best_audio_url)
    
    return best_video_url, best_audio_url

In [39]:
# Get output video and audio filepaths

def get_paths(ytid, ts_start, ts_end):
    
    basename_fmt = '{}_{}_{}'.format(ytid, int(ts_start*1000), int(ts_end*1000))
    video_filepath = os.path.join(videos_path, basename_fmt + '.' + video_container)
    flac_filepath = os.path.join(flac_path, basename_fmt + '.' + audio_container)
    audio_filepath = os.path.join(audio_path, basename_fmt + '.' + 'wav')
    
    return flac_filepath, audio_filepath

### We can then use `ffmpeg` to download and transcode to the format desired:
* To do: remove if file already exists -- this is the -n option in ffmpeg
* To do: can we go directly to wav with ffmeg

In [40]:
# Download the video
def download_video(ffmpeg_path, ts_start, best_video_url, duration, video_container, video_codec, video_filepath):
    
    video_dl_args = [ffmpeg_path, '-n',
        '-ss', str(ts_start),   # The beginning of the trim window
        '-i', best_video_url,   # Specify the input video URL
        '-t', str(duration),    # Specify the duration of the output
        '-f', video_container,  # Specify the format (container) of the video
        '-framerate', '30',     # Specify the framerate
        '-vcodec', video_codec, # Specify the output encoding
        video_filepath]

    proc = sp.Popen(video_dl_args, stdout=sp.PIPE, stderr=sp.PIPE)
    stdout, stderr = proc.communicate()
    if proc.returncode != 0:
        print(stderr)
    else:
        print("Downloaded video to " + video_filepath)

In [41]:
# Download the audio in flac
def download_audio(ffmpeg_path, ts_start, best_audio_url, duration, audio_container, audio_codec, flac_filepath):
    
    audio_dl_args = [ffmpeg_path, '-n',
        '-ss', str(ts_start),    # The beginning of the trim window
        '-i', best_audio_url,    # Specify the input video URL
        '-t', str(duration),     # Specify the duration of the output
        '-f', audio_container, 
        '-vn',                   # Suppress the video stream
        '-ac', '2',              # Set the number of channels
        '-sample_fmt', 's16',    # Specify the bit depth
        '-acodec', audio_codec,  # Specify the output encoding
        '-ar', '44100',          # Specify the audio sample rate
        flac_filepath]

    proc = sp.Popen(audio_dl_args, stdout=sp.PIPE, stderr=sp.PIPE)
    stdout, stderr = proc.communicate()
    if proc.returncode != 0:
        print(stderr)
    else:
        print("Downloaded audio to " + flac_filepath)

In [42]:
# # Download the video (higher quality) separately and merge with the audio
# solo_video_filepath = './solo_' + video_filepath[2:]
# video_dl_args = [ffmpeg_path, '-n',
#     '-ss', str(ts_start),   # The beginning of the trim window
#     '-i', best_video_url,   # Specify the input video URL
#     '-t', str(duration),    # Specify the duration of the output
#     '-f', 'mp4',            # Specify the format (container) of the video
#     '-framerate', '30',     # Specify the framerate
#     '-vcodec', 'h264',      # Specify the output encoding
#     '-crf', '0',            # Lossless
#     '-preset', 'medium',    # Use the medium compression strategy, which
#                             # offers a balance between speed and compression rate
#     solo_video_filepath]

# proc = sp.Popen(video_dl_args, stdout=sp.PIPE, stderr=sp.PIPE)
# stdout, stderr = proc.communicate()
# if proc.returncode != 0:
#     print(stderr)
# else:
#     print("Downloaded video to " + video_filepath)
    
# merge_video_filepath = './merge_' + video_filepath[2:]
# merge_video_dl_args = [ffmpeg_path, '-n',
#     '-i', video_filepath,      # Specify the high quality video path
#     '-i', audio_filepath,      # specify the high quality audio path
#     '-f', 'mp4',               # Specify the format (container) of the video
#     '-framerate', '30',        # Specify the video framerate
#     '-vcodec', 'h264',         # Specify the output video encoding
#     '-acodec', 'aac',          # Specify the output audio encoding
#     '-ac', '2',                # Set the number of channels
#     '-ar', '44100',            # Specify the audio sample rate
#     '-strict', 'experimental', # Needed for merging streams
#     merge_video_filepath]

# proc = sp.Popen(merge_video_dl_args, stdout=sp.PIPE, stderr=sp.PIPE)
# stdout, stderr = proc.communicate()
# if proc.returncode != 0:
#     print(stderr)
# else:
#     print("Merged video at " + merge_video_filepath)

In [43]:
# Convert flac to wav
def conv_wav(ffmpeg_path, flac_filepath, audio_filepath):
    conv_args = [ffmpeg_path, '-n',
        '-i', flac_filepath,   # Specify the input flac filepath
        audio_filepath]

    proc = sp.Popen(conv_args, stdout=sp.PIPE, stderr=sp.PIPE)
    stdout, stderr = proc.communicate()
    if proc.returncode != 0:
        print(stderr)
    else:
        print("Converted flac to wav, saved to " + audio_filepath)

In [44]:
def get_audio_file(video_info):
    '''
    Putting all the functions together 
    '''
    
    ytid, ts_start, ts_end, duration = select_video(video_info)
    best_video_url, best_audio_url = get_urls(ytid)
    flac_filepath, audio_filepath = get_paths(ytid, ts_start, ts_end)
    download_audio(ffmpeg_path, ts_start, best_audio_url, duration, audio_container, audio_codec, flac_filepath)
    conv_wav(ffmpeg_path, flac_filepath, audio_filepath)
    print("")

In [49]:
def download_all_audio(label):
    '''
    Looping the get_audio_file function throughah given list of segments to download audio files.
    Requires a folder to be created of the same name as label in 'data/audio/'
    '''
    
#     audio_path = 'data/audio/' + label  # folder currently needs to be created beforehand
#     print(audio_path)
    video_list = get_video_list(label)
    errors = 0
    for x in video_list:
        try:
            get_audio_file(x)
        except OSError as err:
            errors += 1
            print("OS error: {}".format(err))
            continue
    print('There were {} errors'.format(errors))
    
    #TODO: empty flac folder contents
        

In [51]:
download_all_audio(car)

In [52]:
download_all_audio(outside_rural)